# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier(random_state = 1960)
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RidgeClassifier(random_state=1960)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::any> get_classes(){
		std::vector<std::any> lClasses = { 0, 1, 2 };

		return lClasses;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { 
			"Score_0", "Score_1", "Score_2",
			"Proba_0", "Proba_1", "Proba_2",
			"LogProba_0", "LogProba_1", "LogProba_2",
			"Decision", "DecisionProba" };

		return lOutputs;
	}

	tTable compute_classification_scores(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
		auto lClasses = get_classes();

		std::any score_0 = 0.12726862685332171 * Feature_0 + 0.47083648636124975 * Feature_1 + -0.445366165446255 * Feature_2 + -0.1212031740524525 * Feature_3 + -0.6974613707207697;

		std::any score_1 = -0.027607436590965116 * Feature_0 + -0.8779879015502388 * Feature_1 + 0.3719963526001637 * Featu

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_iris_RidgeClassifier_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_iris_RidgeClassifier_140367567424480.exe /tmp/sklearn2sql_cpp_iris_RidgeClassifier_140367567424480.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,0.95178306206030,-0.74631472744461,-1.20546833461569,,,,,,,0,
1,0.69091109350901,-0.30179928935130,-1.38911180415771,,,,,,,0,
2,0.80416128195522,-0.50907501760317,-1.29508626435205,,,,,,,0,
3,0.65527753754451,-0.34411621326902,-1.31116132427549,,,,,,,0,
4,0.98613984801109,-0.83135277394054,-1.15478707407055,,,,,,,0,
5,1.02044776021643,-1.16076835439741,-0.85967940581902,,,,,,,0,
6,0.82894478259227,-0.72799978956184,-1.10094499303043,,,,,,,0,
7,0.84743593419421,-0.61855555837047,-1.22888037582374,,,,,,,0,
8,0.58019313144622,-0.20019678090079,-1.37999635054543,,,,,,,0,
9,0.70557844300575,-0.26911087367857,-1.43646756932718,,,,,,,0,
10,1.03959433084392,-0.89299490347193,-1.14659942737199,,,,,,,0,
11,

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,-1.203216,-0.606988,0.810205,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
74,74,-0.590155,-0.092786,-0.317060,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
9,9,0.705578,-0.269111,-1.436468,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
88,88,-0.555813,-0.232898,-0.211289,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
25,25,0.614565,-0.230161,-1.384404,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
5,5,1.020448,-1.160768,-0.859679,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
48,48,1.026867,-0.890234,-1.136633,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
117,117,-1.178915,-0.775661,0.954576,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
83,83,-1.127884,0.141589,-0.013706,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
105,105,-1.511654,-0.024422,0.536076,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(clf.decision_function(X), columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_proba = pd.DataFrame(columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,-1.203216,-0.606988,0.810205,NaN,NaN,NaN,NaN,NaN,NaN,2
74,74,-0.590155,-0.092786,-0.317060,NaN,NaN,NaN,NaN,NaN,NaN,1
9,9,0.705578,-0.269111,-1.436468,NaN,NaN,NaN,NaN,NaN,NaN,0
88,88,-0.555813,-0.232898,-0.211289,NaN,NaN,NaN,NaN,NaN,NaN,2
25,25,0.614565,-0.230161,-1.384404,NaN,NaN,NaN,NaN,NaN,NaN,0
5,5,1.020448,-1.160768,-0.859679,NaN,NaN,NaN,NaN,NaN,NaN,0
48,48,1.026867,-0.890234,-1.136633,NaN,NaN,NaN,NaN,NaN,NaN,0
117,117,-1.178915,-0.775661,0.954576,NaN,NaN,NaN,NaN,NaN,NaN,2
83,83,-1.127884,0.141589,-0.013706,NaN,NaN,NaN,NaN,NaN,NaN,1
105,105,-1.511654,-0.024422,0.536076,NaN,NaN,NaN,NaN,NaN,NaN,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,-1.203216,-0.606988,0.810205,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.606988,0.810205,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
74,74,-0.590155,-0.092786,-0.317060,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.092786,-0.317060,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
9,9,0.705578,-0.269111,-1.436468,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.269111,-1.436468,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
88,88,-0.555813,-0.232898,-0.211289,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.232898,-0.211289,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
25,25,0.614565,-0.230161,-1.384404,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.230161,-1.384404,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
5,5,1.020448,-1.160768,-0.859679,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.160768,-0.859679,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
48,48,1.026867,-0.890234,-1.136633,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.890234,-1.136633,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
117,117,-1.178915,-0.775661,0.954576,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.775661,0.954576,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
83,83,-1.127884,0.141589,-0.013706,NaN,NaN,NaN,NaN,NaN,NaN,...,0.141589,-0.013706,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
105,105,-1.511654,-0.024422,0.536076,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.024422,0.536076,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
